In [3]:
import re
import json

class Validator:
    @staticmethod
    def is_valid_email(email):
        pattern = r"^[\w\.\-]+@[\w\.\-]+\.\w+$"
        return bool(re.match(pattern, email))

    @staticmethod
    def is_valid_phone(phone):
        pattern = r'^(010|011|012|015)[0-9]{8}$'
        return bool(re.match(pattern, phone))

    @staticmethod
    def is_non_empty(value):
        return bool(value.strip())

    @staticmethod
    def is_valid_password(password):
        return len(password) >= 8 and re.search(r"[A-Z]", password) and re.search(r"[^a-zA-Z0-9\s]", password)

    @staticmethod
    def is_string(value):
        return isinstance(value, str) and value.isalpha()

In [4]:
class Database:
    @staticmethod
    def load_users(filename="Users.json"):
        try:
            with open(filename, "r") as file:
                users_data = json.load(file)
                return {email: User(**data) for email, data in users_data.items()}
        except FileNotFoundError:
            return {}

    @staticmethod
    def save_users(users, filename="Users.json"):
        users_data = {email: user.to_dict() for email, user in users.items()}
        with open(filename, 'w') as file:
            json.dump(users_data, file, indent=4)


In [5]:
class User:
    def __init__(self, first_name, last_name, email, password, phone):
        if not Validator.is_non_empty(first_name) or not Validator.is_non_empty(last_name):
            raise ValueError("❌ First name or last name cannot be empty")

        if not Validator.is_valid_email(email):
            raise ValueError("❌ Invalid email")

        if not Validator.is_valid_phone(phone):
            raise ValueError("❌ Invalid phone number")

        if not Validator.is_valid_password(password):
            raise ValueError("❌ Invalid password")

        self.first_name = first_name
        self.last_name = last_name
        self.email = email
        self.__password = password
        self.__phone = phone

    @property
    def to_dict(self):
        return {
            'first_name': self.first_name,
            'last_name': self.last_name,
            'email': self.email,
            'password': self.__password,
            'phone': self.__phone
        }

    @property
    def full_name(self):
        return f"{self.first_name} {self.last_name}"

    @property
    def password(self):
        return self.__password

    @password.setter
    def password(self, new_password):
        if Validator.is_valid_password(new_password):
            self.__password = new_password
        else:
            raise ValueError("❌ Invalid password")

    @property
    def phone(self):
        return self.__phone

    @phone.setter
    def phone(self, value):
        if Validator.is_valid_phone(value):
            self.__phone = value
        else:
            raise ValueError("❌ Invalid phone number")

In [6]:
class RegistrationManager:
    def __init__(self):
        self.users_db = Database.load_users()

    def register(self):
        print("Please enter your details to register:")

        while True:
            first_name = input("First Name: ").strip()
            if not Validator.is_string(first_name):
                print("❌ First name must be a valid string (letters only).")
            else:
                break
                
        while True:
            last_name = input("Last Name: ").strip()
            if not Validator.is_string(first_name):
                print("❌ Last name must be a valid string (letters only).")
            else:
                break
        

        while True:
            email = input("Email: ").strip()
            if not Validator.is_valid_email(email):
                print("❌ Invalid email.")
            elif email in self.users_db:
                print("❌ Email already exists.")
            else:
                break

        while True:
            password = input("Password: ").strip()
            if Validator.is_valid_password(password):
                break
            else:
                print("❌ Password too weak.")

        confirm_password = input("Confirm Password: ").strip()
        if password != confirm_password:
            print("❌ Passwords do not match!")
            return None

        while True:
            phone = input("Phone Number: ").strip()
            if Validator.is_valid_phone(phone):
                break
            else:
                print("❌ Invalid phone number!")

        user = User(first_name, last_name, email, password, phone)
        self.users_db[email] = user
        Database.save_users(self.users_db)
        print(f"✅ Welcome {first_name}, registration successful!")
        return user


In [23]:
import getpass
class LoginManager:
    def __init__(self):
        self.users_db = Database.load_users()

    def login(self):
        while True:
            email = input("Enter your email: ").strip()
            password =getpass.getpass("Enter your password: ").strip()

            user = self.users_db.get(email)
            if user and user.password == password:
                print(f"✅ Login successful! Welcome back {user.first_name}!")
                return user
            else:
                print("❌ Invalid email or password. Please try again.")


In [8]:
class ProfileManager:
    def __init__(self, login_manager):
        self.login_manager = login_manager

    def update_profile(self, user):
        if user is None:
            print("❌ You need to log in first.")
            user = self.login_manager.login()
            if user is None:
                return

        new_first_name = input(f"Current First Name: {user.first_name}\nNew First Name: ").strip()
        new_last_name = input(f"Current Last Name: {user.last_name}\nNew Last Name: ").strip()
        new_phone = input(f"Current Phone: {user.phone}\nNew Phone: ").strip()

        if new_first_name:
            user.first_name = new_first_name
        if new_last_name:
            user.last_name = new_last_name
        if Validator.is_valid_phone(new_phone):
            user.phone = new_phone

        users_db = Database.load_users()
        users_db[user.email] = user
        Database.save_users(users_db)
        print("✅ Profile updated successfully!")

In [21]:
class AuthenticationSystem:
    def __init__(self):
        self.login_manager = LoginManager()
        self.registration_manager = RegistrationManager()
        self.profile_manager = ProfileManager(self.login_manager)
        self.logged_in_user = None
        self.authentication_menu()

    def authentication_menu(self):
        while True:
            print("\n🔐 Authentication System")
            print("1. Register")
            print("2. Log In")
            print("3. Update Profile")
            print("4. Back to Main Menu")

            choice = input("Choose an option: ").strip()

            if choice == '1':
                self.logged_in_user = self.registration_manager.register()

            elif choice == '2':
                self.logged_in_user = self.login_manager.login()
                
            elif choice == '3':
                self.profile_manager.update_profile(self.logged_in_user)

            elif choice == '4':
                print("🔙 Returning to main menu...\n")
                return self.logged_in_user.email
            else:
                print("❌ Invalid choice. Please try again.")
        